### **dual_ppo**

根据论文以及一些建议，考虑使用**dual_ppo**优化算法，在$A<0$时给一个下限$c \times A$，抑制负优势样本对更新的过度惩罚，当$A \ge 0$时保持原ppo



### **具体代码**


在`conf/conf.py`中加入如下的参数控制`dual_ppo`
```python
USE_DUAL_CLIP_PPO = True
DUAL_CLIP_C = 2.0
```

在`model/model.py`中修改为：
```python
surr1 = clip_ratio * advantage
surr2 = ratio.clamp(1.0 - self.clip_param, 1.0 + self.clip_param) * advantage
ppo_objective = torch.minimum(surr1, surr2)
if self.use_dual_clip_ppo:
    dual_clip_objective = self.dual_clip_c * advantage
    surrogate = torch.where(advantage >= 0, ppo_objective,
                            torch.maximum(ppo_objective, dual_clip_objective))
else:
    surrogate = ppo_objective

temp_policy_loss = -torch.sum(
    surrogate * (weight_list[task_index].float()) * 1
) / torch.maximum(torch.sum((weight_list[task_index].float()) * 1), torch.tensor(1.0))
```